# OpenSatCom Quickstart: Snapshot Link Budget

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jman4162/opensatcom/blob/main/notebooks/01_quickstart.ipynb)

This notebook demonstrates a complete end-to-end snapshot link budget evaluation:
1. Define terminals (satellite + ground station)
2. Configure antenna, RF chain, and propagation
3. Evaluate the link budget
4. Visualize the results

In [ ]:
# Install opensatcom (uncomment for Colab)
# !pip install -q opensatcom

In [ ]:
import numpy as np
from opensatcom.core.models import (
    LinkInputs, RFChainModel, Scenario, Terminal, PropagationConditions
)
from opensatcom.antenna.parametric import ParametricAntenna
from opensatcom.propagation import FreeSpacePropagation, CompositePropagation
from opensatcom.link.engine import DefaultLinkEngine
from opensatcom.geometry.slant import slant_range_m

## 1. Define Terminals

A GEO satellite at 35,786 km altitude and a ground station at sea level.

In [ ]:
satellite = Terminal(
    name="GEO-Sat",
    lat_deg=0.0,
    lon_deg=0.0,
    alt_m=35_786_000.0,
)

ground_station = Terminal(
    name="Ground-Station",
    lat_deg=38.9,
    lon_deg=-77.0,
    alt_m=0.0,
    system_noise_temp_k=290.0,
)

print(f"Satellite: {satellite.name} at {satellite.alt_m/1e3:.0f} km")
print(f"Ground:    {ground_station.name} at ({ground_station.lat_deg}, {ground_station.lon_deg})")

## 2. Configure Scenario & RF Chain

In [ ]:
scenario = Scenario(
    name="Ku-band Downlink",
    direction="downlink",
    freq_hz=12e9,
    bandwidth_hz=36e6,
    polarization="RHCP",
    required_metric="ebn0_db",
    required_value=5.0,
)

rf_chain = RFChainModel(
    tx_power_w=100.0,
    tx_losses_db=1.5,
    rx_noise_temp_k=75.0,
)

print(f"Frequency: {scenario.freq_hz/1e9:.1f} GHz")
print(f"Bandwidth: {scenario.bandwidth_hz/1e6:.0f} MHz")
print(f"TX Power:  {rf_chain.tx_power_w} W")

## 3. Configure Antennas

In [ ]:
tx_antenna = ParametricAntenna(gain_dbi=36.0)
rx_antenna = ParametricAntenna(gain_dbi=38.0)

print(f"TX Antenna Gain: {tx_antenna._gain_dbi:.1f} dBi")
print(f"RX Antenna Gain: {rx_antenna._gain_dbi:.1f} dBi")

## 4. Build Link Inputs & Evaluate

In [ ]:
link_inputs = LinkInputs(
    tx_terminal=satellite,
    rx_terminal=ground_station,
    scenario=scenario,
    tx_antenna=tx_antenna,
    rx_antenna=rx_antenna,
    propagation=FreeSpacePropagation(),
    rf_chain=rf_chain,
)

elev_deg = 30.0
range_m = slant_range_m(ground_station.alt_m, satellite.alt_m, elev_deg)

engine = DefaultLinkEngine()
result = engine.evaluate_snapshot(
    elev_deg=elev_deg, az_deg=0.0, range_m=range_m,
    inputs=link_inputs, cond=PropagationConditions(),
)

print(f"\n{'='*40}")
print(f"LINK BUDGET RESULTS")
print(f"{'='*40}")
print(f"EIRP:       {result.eirp_dbw:.2f} dBW")
print(f"Path Loss:  {result.path_loss_db:.2f} dB")
print(f"G/T:        {result.gt_dbk:.2f} dB/K")
print(f"C/N0:       {result.cn0_dbhz:.2f} dB-Hz")
print(f"Eb/N0:      {result.ebn0_db:.2f} dB")
print(f"Margin:     {result.margin_db:.2f} dB")

## 5. Margin vs Elevation Sweep

In [ ]:
elevations = np.linspace(5, 90, 50)
margins = []

for el in elevations:
    rng = slant_range_m(ground_station.alt_m, satellite.alt_m, el)
    out = engine.evaluate_snapshot(el, 0.0, rng, link_inputs, PropagationConditions())
    margins.append(out.margin_db)

margins = np.array(margins)

In [ ]:
from opensatcom.viz.timeline import plot_link_margin_timeline

fig = plot_link_margin_timeline(
    elevations, margins,
    title="Link Margin vs Elevation Angle"
)
fig.update_layout(xaxis_title="Elevation (deg)")
fig.show()

## 6. Add Rain Attenuation

Compare FSPL-only vs FSPL + rain attenuation.

In [ ]:
from opensatcom.propagation import RainAttenuationP618

rain_model = RainAttenuationP618(rain_rate_mm_per_hr=25.0, availability_target=0.99)
composite = CompositePropagation([FreeSpacePropagation(), rain_model])

link_inputs_rain = LinkInputs(
    tx_terminal=satellite, rx_terminal=ground_station,
    scenario=scenario, tx_antenna=tx_antenna, rx_antenna=rx_antenna,
    propagation=composite, rf_chain=rf_chain,
)

margins_rain = []
for el in elevations:
    rng = slant_range_m(ground_station.alt_m, satellite.alt_m, el)
    out = engine.evaluate_snapshot(el, 0.0, rng, link_inputs_rain, PropagationConditions())
    margins_rain.append(out.margin_db)

margins_rain = np.array(margins_rain)

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=elevations, y=margins, name="FSPL only", line=dict(color="blue")))
fig.add_trace(go.Scatter(x=elevations, y=margins_rain, name="FSPL + Rain", line=dict(color="red")))
fig.add_hline(y=0, line_dash="dash", line_color="gray")
fig.update_layout(title="Margin: Clear Sky vs Rain", xaxis_title="Elevation (deg)", yaxis_title="Margin (dB)", template="plotly_white")
fig.show()

---

**Next steps:** See `02_mission_simulation.ipynb` for time-series analysis.